# Recolección y Exploración del Corpus de Noticias

## Introducción
El análisis de texto es una técnica fundamental en el procesamiento del lenguaje natural (NLP), utilizada para extraer información significativa de grandes volúmenes de datos textuales. En este caso, trabajaremos con un conjunto de noticias provenientes de un archivo de datos en Excel, con el objetivo de realizar una carga, exploración, filtrado y análisis del corpus de noticias. Además, se implementarán técnicas de preprocesamiento como limpieza del texto, tokenización y padding, permitiendo mejorar la calidad de los datos procesados para su uso en modelos de aprendizaje profundo.

### Este cuaderno Jupyter se enfocará en:

1. Carga y visualización del corpus 
* Importación del archivo de noticias en un DataFrame de Pandas.
* Inspección de la estructura y calidad de los datos.
* Identificación de valores nulos o inconsistencias en las noticias.

2. Exploración del corpus
* Análisis estadístico del conjunto de datos.
* Conteo de la cantidad de noticias por categoría.
* Cálculo de la longitud promedio de los artículos en términos de palabras.

3. Palabras más frecuentes
* Extracción de todas las palabras del corpus de noticias.
* Conteo de la frecuencia de cada palabra en el conjunto de datos.
* Identificación de las palabras más utilizadas en los artículos.

4. Procesamiento del texto
* Limpieza del texto: Conversión a minúsculas, eliminación de caracteres especiales y números.
* Tokenización: Conversión del texto en secuencias numéricas.
* Padding: Normalización de las secuencias a una longitud fija para modelos de aprendizaje profundo.

5. Filtrado de Etiquetas Más Relevantes
* Identificación de las etiquetas más comunes en el corpus.
* Selección de las 4 etiquetas más frecuentes para reducir el ruido en el análisis.
* Filtrado del conjunto de datos para incluir solo las categorías más representativas.

## Importancia del Corpus de Noticias
El análisis de este corpus nos permitirá identificar tendencias, patrones lingüísticos y relaciones entre palabras, lo cual es útil para aplicaciones como:

* Análisis de sentimientos: Determinar la polaridad de las noticias.
* Clasificación de noticias: Organizar artículos en categorías relevantes.
* Extracción de palabras clave: Identificar términos importantes dentro del corpus.
* Modelado de temas: Descubrir patrones y agrupaciones temáticas en los datos.
* Filtrado de etiquetas relevantes: Reducir el ruido en la clasificación de noticias y enfocarse en las categorías de mayor interés.

# 1. Carga y Visualización del Corpus

In [36]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ojito\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ojito\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Este fragmento de código descarga dos recursos clave de la biblioteca NLTK (Natural Language Toolkit), que es una de las herramientas más utilizadas para el procesamiento del lenguaje natural (NLP):

1. nltk.download('punkt'):
* Descarga el paquete Punkt, que contiene un modelo de tokenización preentrenado.
* Se utiliza para dividir un texto en oraciones o palabras, facilitando el análisis del corpus.

2. nltk.download('stopwords'):
* Descarga una lista de stopwords (palabras vacías o de poco significado, como "el", "la", "de", etc.).
* Estas palabras suelen eliminarse en tareas de análisis de texto, ya que no aportan mucho significado a nivel de contenido.

In [38]:
# Importar las bibliotecas necesarias
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

# Descargar recursos de NLTK
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

# Cargar el archivo de datos
file_path = r'C:\Users\ojito\Gerencia de Proyectos 2\Noticias.xlsx'

data = pd.read_excel(file_path)

# Mostrar una vista previa de los datos

data.shape

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ojito\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ojito\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(14396, 6)

Este fragmento de código se encarga de importar bibliotecas, descargar recursos de NLP, cargar el dataset y mostrar su estructura.

1. Importación de bibliotecas
* pandas: Para la manipulación y análisis del dataset.
* nltk.tokenize.word_tokenize: Para dividir el texto en palabras.
* string y re: Para manipulación de texto y expresiones regulares.
* TfidfVectorizer (Scikit-learn): Para la vectorización de texto usando TF-IDF, una técnica que mide la importancia de cada palabra en un documento.
* Word2Vec (Gensim): Para la representación de palabras en vectores numéricos basados en su contexto en el corpus.

2. Descarga de recursos de NLTK
* punkt: Para tokenización en NLTK.
* stopwords: Contiene palabras vacías en varios idiomas.

3. Carga del dataset
* Carga el archivo de Excel en un DataFrame de Pandas.
* Se asume que el script se ejecuta en Google Colab (por el uso de '/content/' como ruta).

4. Visualización de la estructura de los datos
* shape devuelve la forma del dataset en (filas, columnas), permitiendo conocer el tamaño del corpus de noticias.

In [40]:
if data.empty:
    print("Error: El archivo está vacío o no se pudo cargar.")
else:
    print(f"Dataset cargado correctamente con {data.shape[0]} filas y {data.shape[1]} columnas.")

Dataset cargado correctamente con 14396 filas y 6 columnas.


Mostramos las primeras filas para entender la estructura

In [25]:
data.head()

,Columna1,Enlaces,Título,info,contenido,Etiqueta
0,0,https://www.eltiempo.com/agresion-contra-un-op...,Operador de grúa quedó inconsciente tras agres...,El conductor de una moto le lanzó el casco y p...,Las autoridades están buscando al conductor de...,colombia
1,1,https://www.eltiempo.com/archivo/documento/CMS...,"Usaquén, primera en infracciones por mal parqueo",La localidad ocupa el primer lugar en comparen...,"""Los andenes son para los peatones"", reclama e...",archivo
2,2,https://www.eltiempo.com/archivo/documento/CMS...,'Me atracaron y vi un arma que me heló la sang...,Un ciudadano relata cómo cuatro hombres lo rob...,A las 7 de la noche me había quedado de encont...,archivo
3,3,https://www.eltiempo.com/archivo/documento/CMS...,"Escoltas mal estacionados, dolor de cabeza de ...",Las zonas de restaurantes se convierten en par...,Atravesados. Eso es lo que se les pasa por la ...,archivo
4,4,https://www.eltiempo.com/archivo/documento/CMS...,Radicado primer proyecto que autorizaría union...,"El representante de 'la U', Miguel Gómez, dijo...",“Estamos proponiendo la figura de un contrato ...,archivo


# 2. Exploración del Corpus

In [44]:
# Cantidad de noticias por categoría
print("\n📌 Cantidad de noticias por etiqueta:")
print(data["Etiqueta"].value_counts())

# Longitud promedio de los artículos (usando 'contenido' en lugar de 'contenido_preprocesado')
data["longitud"] = data["contenido"].dropna().apply(lambda x: len(str(x).split()))
print("\n📊 Longitud promedio de los artículos:", data["longitud"].mean())

# Palabras más frecuentes en el corpus (usando 'contenido' en lugar de 'contenido_preprocesado')
from collections import Counter

# Concatenar todos los textos en un solo string
texto_completo = " ".join(data["contenido"].dropna())
palabras = texto_completo.split()
conteo_palabras = Counter(palabras)


📌 Cantidad de noticias por etiqueta:
Etiqueta
archivo                 9187
colombia                 934
deportes                 727
opinion                  532
mundo                    446
cultura                  430
economia                 367
justicia                 343
bogota                   311
vida                     268
politica                 252
tecnosfera               214
salud                    106
historias-el-tiempo       57
mundial                   47
contenido-comercial       34
elecciones                33
unidad-investigativa      27
podcast                   20
foro-w                    18
bocas                     15
carrusel                   8
datos                      7
lecturas-dominicales       6
mas-contenido              4
especiales                 3
Name: count, dtype: int64

📊 Longitud promedio de los artículos: 531.6849614208764


1. Cantidad de noticias por categoría (Etiqueta)
* Se cuenta cuántas noticias hay en cada categoría y se imprime el resultado.
  
2. Longitud promedio de los artículos (contenido)
* Se calcula la cantidad promedio de palabras en los artículos.
* Se usa .dropna() para evitar errores con valores nulos.
* Se utiliza split() para contar el número de palabras en cada artículo.

# 3. Palabras Más Frecuentes

In [48]:
from collections import Counter

# Obtener todas las palabras en una sola lista (usando 'contenido' en lugar de 'tokens_sin_stopwords')
todas_las_palabras = [word for text in data["contenido"].dropna() for word in text.split()]

# Contar la frecuencia de cada palabra
conteo_palabras = Counter(todas_las_palabras)

# Mostrar las 10 palabras más comunes
print("\n🔝 Palabras más frecuentes en el corpus:")
print(conteo_palabras.most_common(10))


🔝 Palabras más frecuentes en el corpus:
[('de', 496879), ('la', 253580), ('en', 212954), ('que', 190982), ('el', 181023), ('y', 161176), ('a', 135845), ('del', 112492), ('los', 92772), ('con', 85706)]


Este fragmento de código cuenta las palabras más frecuentes en el corpus de noticias utilizando la columna contenido. Aquí está el desglose de lo que hace:

1. Obtener todas las palabras en una sola lista
* Se itera sobre la columna contenido, asegurando que no haya valores nulos con .dropna().
* Cada artículo de noticias se divide en palabras usando .split(), generando una lista de todas las palabras.

2. Contar la frecuencia de cada palabra
* Se utiliza la clase Counter de la biblioteca collections para contar cuántas veces aparece cada palabra en el conjunto de datos.

3. Mostrar las 10 palabras más comunes
* Se usa most_common(10) para obtener e imprimir las 10 palabras con mayor frecuencia en el corpus.

# Limpieza, tokenización y vectorización con LSTM

# 1. Importación de librerías

In [51]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import re
import pickle

* Tokenizer: Se utiliza para convertir el texto en secuencias numéricas.
* pad_sequences: Ayuda a normalizar la longitud de las secuencias, agregando padding si es necesario.
* numpy: Para manipular datos numéricos.
* re: Librería de expresiones regulares para limpiar texto.
* pickle: Para guardar el tokenizer y reutilizarlo después.

# 2. Definir parámetros

In [53]:
# Parámetros
MAX_NB_WORDS = 5000  # Número máximo de palabras en el vocabulario
MAX_SEQUENCE_LENGTH = 200  # Longitud máxima de las secuencias
EMBEDDING_DIM = 100  # Dimensión de los embeddings

Estos valores se utilizan para configurar el procesamiento del texto:

* MAX_NB_WORDS = 5000: Solo se considerarán las 5000 palabras más frecuentes en el vocabulario.
* MAX_SEQUENCE_LENGTH = 200: Todas las secuencias serán truncadas o rellenadas hasta 200 palabras.
* EMBEDDING_DIM = 100: Cada palabra en el modelo LSTM se representará con un vector de 100 dimensiones.

# 3. Función para limpiar el texto

In [55]:
import re
from nltk.corpus import stopwords

# Lista de stopwords en español
stop_words = set(stopwords.words("spanish"))

# Función para limpiar el texto y eliminar stopwords
def limpiar_texto(texto):
    if isinstance(texto, float):  # Si es NaN, lo convertimos a una cadena vacía
        return ""
    texto = str(texto).lower()  # Convertir a minúsculas
    texto = re.sub(r'[^a-záéíóúüñ ]', '', texto)  # Eliminar caracteres especiales y números
    palabras = texto.split()  # Tokenizar el texto dividiéndolo por espacios
    palabras_filtradas = [word for word in palabras if word not in stop_words]  # Eliminar stopwords
    return " ".join(palabras_filtradas)  # Volver a unir las palabras en una cadena

# Aplicar la limpieza al conjunto de datos
data["contenido_limpio"] = data["contenido"].astype(str).apply(limpiar_texto)

* Convierte texto a minúsculas
* Elimina caracteres especiales y números
* Elimina stopwords en español
* Reconstruye el texto limpio
* Aplica la limpieza a todo el DataFrame

In [57]:
from collections import Counter

# Obtener todas las palabras en una sola lista después de la limpieza
todas_las_palabras = [word for text in data["contenido_limpio"].dropna() for word in text.split()]

# Contar la frecuencia de cada palabra
conteo_palabras = Counter(todas_las_palabras)

# Mostrar las 10 palabras más comunes después de eliminar stopwords
print("\n🔝 Palabras más frecuentes en el corpus después de eliminación de stopwords:")
print(conteo_palabras.most_common(10))


🔝 Palabras más frecuentes en el corpus después de eliminación de stopwords:
[('pm', 72074), ('colombia', 37084), ('valle', 28618), ('manizales', 22344), ('coronavirus', 18701), ('mil', 18185), ('mercados', 17190), ('barranquilla', 15978), ('cerca', 15634), ('cauca', 14515)]


* Counter es una herramienta de la librería estándar collections de Python: Se usa para contar la frecuencia de elementos dentro de una lista.
* Extrae todas las palabras de la columna "contenido_limpio" del DataFrame data.
* .dropna(): Se usa para eliminar valores nulos (NaN) y evitar errores.
* Tokenización simple (split()): Divide cada texto en palabras individuales.
* Comprensión de listas (list comprehension): Recorre todas las filas y extrae cada palabra.
* Usa Counter para contar cuántas veces aparece cada palabra en el corpus.

 # 4. Aplicar limpieza a los datos

In [59]:
# Aplicar limpieza asegurando que todos los valores sean cadenas de texto
data["contenido_limpio"] = data["contenido"].astype(str).apply(limpiar_texto)

* Aplica la función limpiar_texto() a la columna contenido del dataset.
* Asegura que todos los valores son cadenas (astype(str)) antes de aplicar la limpieza.
* Guarda el resultado en la nueva columna contenido_limpio.

# 5. Función para tokenizar el texto

In [61]:
# Función de tokenización
def tokenizar_texto(textos):
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS, oov_token="<OOV>")
    tokenizer.fit_on_texts(textos)
    sequences = tokenizer.texts_to_sequences(textos)
    return tokenizer, sequences

Esta función:

* Crea un tokenizador que usa MAX_NB_WORDS=5000 palabras más frecuentes.
* Usa el token <OOV> (Out of Vocabulary) para palabras desconocidas.
* Convierte el texto en secuencias de números, donde cada palabra se convierte en su índice en el vocabulario.

# 6. Aplicar tokenización

In [63]:
# Aplicar tokenización
tokenizer, sequences = tokenizar_texto(data["contenido_limpio"])

* Tokeniza el texto limpio, convirtiéndolo en secuencias de números.
* Cada palabra tiene un número asociado según el tokenizer.

# 7. Padding para normalizar las secuencias

In [65]:
# Padding para hacer las secuencias del mismo tamaño
def aplicar_padding(sequences, max_length):
    return pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

Esta función:

* Asegura que todas las secuencias tengan el mismo largo máximo (MAX_SEQUENCE_LENGTH = 200).
* Usa padding='post', lo que significa que si la secuencia es más corta, se agregan ceros al final.
* Usa truncating='post', lo que significa que si la secuencia es más larga, se corta al final.

# 8. Aplicar padding

In [67]:
X = aplicar_padding(sequences, MAX_SEQUENCE_LENGTH)

* Convierte las secuencias tokenizadas en matrices listas para LSTM.
* X ahora es una matriz (número de ejemplos, 200).
* Ejemplo de salida:

Antes del padding:
[5, 87, 302, 15, 928]

Después del padding:
[5, 87, 302, 15, 928, 0, 0, 0, 0, ..., 0]  (hasta 200)

# 9. Imprimir información sobre los datos procesados

In [69]:
# Imprimir información sobre los datos tokenizados
print(f"🔢 Total de palabras en el vocabulario: {len(tokenizer.word_index)}")
print(f"🔠 Forma de una secuencia de entrada: {X.shape}")

🔢 Total de palabras en el vocabulario: 123089
🔠 Forma de una secuencia de entrada: (14396, 200)


* Cantidad de palabras en el vocabulario (tokenizer.word_index).
* Dimensiones de las secuencias (X.shape).

# 10. Guardar el tokenizer para uso futuro

In [71]:
# Guardar tokenizer para uso futuro
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

* Guarda el tokenizer en un archivo pickle (.pkl) para reutilizarlo sin volver a tokenizar el texto.

# 11. Muestra las primeras 5 secuencias tokenizadas

In [73]:
# Visualizar algunas secuencias tokenizadas
print("\nprimeras 5 secuencias procesadas:")
print(X[:5])


primeras 5 secuencias procesadas:
[[ 180 1753 1680 2827 2178 4278  169  889    1 3790    1  201    1 1127
   297 1014  114 4666  908  544 1806  108    1  201    1  807    1   10
  1074 1402 3870 3628    1  527    1  674    1    1 2990  609  234 1680
  2827    1 4278 3790  133 2179  601 1010 1369    1  934    1  608    1
  4397    1 1127 1014   10  297    1    1    1    1  149    1 1080 2797
   938 3790 2138  113 2959    1 2827    1  149  890  420    1    1  180
  1457  133    1 2827    1 1768  489    1 1705 1229    1 4666 1954    1
    24  453    1  340  404  640  587  170   90  485 3492   99  103  201
     1    1 1464 1052   99    1  242   76    4   11    2   15   10    7
     8   16    4   14   93   83    7    8   41  110   77    6    3    2
    12   58   64   65   38  160  135  190  244  277  312  125   55   68
     2  132  143  141   67  146   55   68    9    2  111  145   91   95
   140  112  188    2  133  241  192  187  325  207    0    0    0    0
     0    0    0    0    0   

* Cada fila representa un texto convertido en números y normalizado para usarlo en LSTM.

# 12. Definir la función obtener_etiquetas_relevantes

In [75]:
def obtener_etiquetas_relevantes(data, columna_etiqueta="Etiqueta", top_n=4):
    etiquetas_comunes = Counter(data[columna_etiqueta]).most_common(top_n)
    etiquetas_relevantes = [etiqueta[0] for etiqueta in etiquetas_comunes]
    return etiquetas_relevantes 

¿Qué hace esta función?

* Cuenta la frecuencia de cada etiqueta en la columna "Etiqueta" utilizando Counter de collections.
* Obtiene las top_n etiquetas más comunes con .most_common(top_n).
* top_n=4 significa que se seleccionarán las 4 etiquetas más frecuentes.
* Extrae solo los nombres de las etiquetas en una lista.
* Devuelve la lista de etiquetas más comunes.

# 13. Obtener y mostrar las etiquetas más relevantes

In [77]:
etiquetas_mas_relevantes = obtener_etiquetas_relevantes(data)
print(f"\n🎯 Etiquetas más relevantes: {etiquetas_mas_relevantes}")


🎯 Etiquetas más relevantes: ['archivo', 'colombia', 'deportes', 'opinion']


* Se llama a la función obtener_etiquetas_relevantes(data).
* Se almacenan las etiquetas más comunes en etiquetas_mas_relevantes.
* Se imprimen las etiquetas seleccionadas.

# 14. Filtrar el dataset para conservar solo esas etiquetas

In [79]:
data_filtrada = data[data["Etiqueta"].isin(etiquetas_mas_relevantes)]
print(f"\n📌 Nuevo tamaño del dataset después del filtro: {data_filtrada.shape}")


📌 Nuevo tamaño del dataset después del filtro: (11380, 8)


* Esto indica que, después del filtrado, el dataset ahora tiene 11380 filas y 8 columnas.